In [8]:
import re

# Read the MDscan output file
with open('Results/results_md/Ada_1.txt', 'r') as file:
    mdscan_output = file.read()

# Use regex patterns to extract the motif information
motif_pattern = r"Motif (\d+):\s+Wid (\d+);\s+Score ([\d.]+);\s+Sites (\d+);\s+Con (\w+);\s+RCon (\w+)"
motif_info = re.findall(motif_pattern, mdscan_output)

# # Use regex patterns to extract the motif alignment matrix
# alignment_pattern = r"\d+\s+(\d+\.\d+)\s+(\d+\.\d+)\s+(\d+\.\d+)\s+(\d+\.\d+)\s+(\w+)\s+(\w+)\s+(\w+)\s+(\w+)"
# alignment_matrix = re.findall(alignment_pattern, mdscan_output)

# Use regex patterns to extract the site information
site_pattern = r">(\d+-\d+-(?:forward|reverse))\s+Len\s+\d+\s+Site\s+#\d+\s+(f|r)\s+(\d+)\n([ACGT]+)"
site_info = re.findall(site_pattern, mdscan_output)

# # Print the extracted motif information
# print("Motif Information:")
# for motif in motif_info:
#     motif_number, motif_width, motif_score, site_count, consensus, reverse_consensus = motif
#     print(f"Motif Number: {motif_number}")
#     print(f"Motif Width: {motif_width}")
#     print(f"Motif Score: {motif_score}")
#     print(f"Number of Sites: {site_count}")
#     print(f"Consensus Sequence: {consensus}")
#     print(f"Reverse Consensus Sequence: {reverse_consensus}\n")

# # Print the extracted motif alignment matrix
# print("Motif Alignment Matrix:")
# for alignment in alignment_matrix:
#     position_values = alignment[:4]
#     consensus, reverse_consensus, degenerate, reverse_degenerate = alignment[4:]
#     print("Position Values:", position_values)
#     print("Consensus: ", consensus)
#     print("Reverse Consensus: ", reverse_consensus)
#     print("Degenerate: ", degenerate)
#     print("Reverse Degenerate: ", reverse_degenerate)
#     print()


FileNotFoundError: [Errno 2] No such file or directory: 'Results/results_md/Ada_1.txt'

In [7]:
site_info

[('209398-209425-forward', 'f', '151', 'AAGCGCCGCTGGCGG'),
 ('209398-209425-forward', 'r', '258', 'CGCCATCGCTTCCGG'),
 ('209398-209425-forward', 'f', '166', 'CTGAAGCGATGGGTA'),
 ('209398-209425-forward', 'f', '229', 'CGGAACCACTGGGTG'),
 ('209398-209425-forward', 'f', '259', 'CGGAAGCGATGGCGG'),
 ('209398-209425-forward', 'f', '286', 'CGCAACTGCTGGCGG'),
 ('2145603-2145630-reverse', 'f', '348', 'TGGAAACGGTCGCGG'),
 ('2308475-2308502-reverse', 'r', '409', 'CGCAAAGCATGTCTG'),
 ('2308475-2308502-reverse', 'f', '36', 'GGAAGCCGATGCCTG'),
 ('2308475-2308502-reverse', 'r', '378', 'GGCAAAAGATGCCTG'),
 ('2308475-2308502-reverse', 'r', '351', 'CGAAAACGAATTCGC'),
 ('2308475-2308502-reverse', 'r', '345', 'CGAATTCGCCGTCGG'),
 ('2308475-2308502-reverse', 'f', '104', 'GGGAAGGGCTGGCGG'),
 ('2308475-2308502-reverse', 'f', '135', 'AGAAGGCGATAGCTT'),
 ('209398-209425-forward', 'r', '286', 'CCGCCAGCAGTTGCG'),
 ('209398-209425-forward', 'r', '271', 'CTTTCAACGATGCCG'),
 ('209398-209425-forward', 'r', '259', 'C